### Laion 2b es4.5

In [3]:
import datasets
from datasets import load_dataset

In [4]:
ds = load_dataset("parquet", data_dir="/home/yx/datasets/diffusion/laion_2b_en_as4.5plus")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/default-94e46530464c81b4/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
type(ds['train'])

datasets.arrow_dataset.Dataset

In [12]:
len(ds['train'])

21440657

In [68]:
ds['train']['WIDTH']

[340.0,
 220.0,
 259.0,
 1188.0,
 299.0,
 720.0,
 413.0,
 193.0,
 800.0,
 800.0,
 240.0,
 300.0,
 300.0,
 875.0,
 536.0,
 2000.0,
 250.0,
 1000.0,
 236.0,
 242.0,
 356.0,
 200.0,
 225.0,
 250.0,
 600.0,
 180.0,
 270.0,
 280.0,
 1200.0,
 400.0,
 960.0,
 450.0,
 640.0,
 450.0,
 1169.0,
 150.0,
 500.0,
 180.0,
 220.0,
 800.0,
 230.0,
 300.0,
 260.0,
 480.0,
 400.0,
 250.0,
 200.0,
 240.0,
 236.0,
 638.0,
 220.0,
 400.0,
 585.0,
 720.0,
 1024.0,
 177.0,
 800.0,
 500.0,
 236.0,
 300.0,
 240.0,
 630.0,
 1200.0,
 200.0,
 604.0,
 500.0,
 300.0,
 220.0,
 653.0,
 707.0,
 414.0,
 347.0,
 615.0,
 500.0,
 640.0,
 281.0,
 450.0,
 632.0,
 150.0,
 640.0,
 640.0,
 220.0,
 340.0,
 192.0,
 360.0,
 522.0,
 500.0,
 300.0,
 1200.0,
 800.0,
 325.0,
 193.0,
 209.0,
 216.0,
 410.0,
 800.0,
 300.0,
 225.0,
 1000.0,
 962.0,
 960.0,
 667.0,
 850.0,
 110.0,
 331.0,
 225.0,
 1050.0,
 583.0,
 1088.0,
 160.0,
 512.0,
 600.0,
 300.0,
 220.0,
 1024.0,
 272.0,
 480.0,
 500.0,
 314.0,
 1142.0,
 223.0,
 650.0,
 480.0,
 41

In [70]:
max(ds['train']['punsafe'])

1.0

In [71]:
min(ds['train']['AESTHETIC_SCORE'])

4.5

In [10]:
ds['train'].column_names

['URL',
 'TEXT',
 'WIDTH',
 'HEIGHT',
 'similarity',
 'punsafe',
 'pwatermark',
 'AESTHETIC_SCORE',
 'hash',
 '__index_level_0__']

In [11]:
sample0 = ds['train'].__getitem__(0)

{'URL': 'https://img1.etsystatic.com/186/0/5531524/il_340x270.1310915051_5kax.jpg',
 'TEXT': 'Vintage Green Glass Swag Hanging Lamp Mid Century',
 'WIDTH': 340.0,
 'HEIGHT': 270.0,
 'similarity': 0.3491742014884949,
 'punsafe': 8.991600225272123e-06,
 'pwatermark': 0.14151343703269958,
 'AESTHETIC_SCORE': 4.751741409301758,
 'hash': 6170073934346815248,
 '__index_level_0__': 0}

In [63]:
ds['train'].__getitem__(100)

{'URL': 'https://cdn-uploads.fresha.com/location-profile-images/356736/347276/small_de30404c-53d3-44e7-82f4-ccb2998190d1.jpg',
 'TEXT': 'K.T Beauty Therapies',
 'WIDTH': 960.0,
 'HEIGHT': 540.0,
 'similarity': 0.3616345226764679,
 'punsafe': 0.00014868378639221191,
 'pwatermark': 0.30610936880111694,
 'AESTHETIC_SCORE': 5.360263347625732,
 'hash': -3596457088321009045,
 '__index_level_0__': 158}

### Laion-art

In [15]:
art = load_dataset("parquet", data_dir="/home/yx/datasets/diffusion/laion_art")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/default-d61c732c98ca6f56/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
art['train'].__getitem__(0)

{'URL': 'https://www.advocate-art.com/system/ART/Modules/Application/Images/Image/images/000/011/121/artistique_half/VML175107.jpg?6f71107bead7921d03fa7dc3e4ac4b9a8f24dfd3d823d512d7618c06e4059513',
 'TEXT': 'Christmas Shopping Copy',
 'WIDTH': 850,
 'HEIGHT': 850,
 'similarity': 0.2666246294975281,
 'LANGUAGE': 'nolang',
 'hash': -3604776403351267688,
 'pwatermark': 0.0396263524889946,
 'punsafe': 0.00027811527252197266,
 'aesthetic': 8.352225303649902}

In [17]:
len(art['train'])

8070941

In [25]:
8070941 * 0.1

807094.1000000001

### Filtering and Download

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import rand

#def filter_parquet(data_path_or_dir, width=None, height=None):
data_path_or_dir='/data3/datasets/laion_art_metadata'
filter_width=512
filter_height=512
num_repartitions = 1
output_dir = '/data3/datasets/laion_art_filtered'

spark = SparkSession.builder.config("spark.driver.memory", "16G") .master("local[16]").appName('spark-stats').getOrCreate() 
df = spark.read.parquet(data_path_or_dir)



In [3]:
df.schema

StructType([StructField('URL', StringType(), True), StructField('TEXT', StringType(), True), StructField('WIDTH', IntegerType(), True), StructField('HEIGHT', IntegerType(), True), StructField('similarity', DoubleType(), True), StructField('LANGUAGE', StringType(), True), StructField('hash', LongType(), True), StructField('pwatermark', FloatType(), True), StructField('punsafe', FloatType(), True), StructField('aesthetic', FloatType(), True)])

In [4]:
df.printSchema()

root
 |-- URL: string (nullable = true)
 |-- TEXT: string (nullable = true)
 |-- WIDTH: integer (nullable = true)
 |-- HEIGHT: integer (nullable = true)
 |-- similarity: double (nullable = true)
 |-- LANGUAGE: string (nullable = true)
 |-- hash: long (nullable = true)
 |-- pwatermark: float (nullable = true)
 |-- punsafe: float (nullable = true)
 |-- aesthetic: float (nullable = true)



In [5]:
df.show()

+--------------------+--------------------+-----+------+-------------------+--------+--------------------+-----------+------------+---------+
|                 URL|                TEXT|WIDTH|HEIGHT|         similarity|LANGUAGE|                hash| pwatermark|     punsafe|aesthetic|
+--------------------+--------------------+-----+------+-------------------+--------+--------------------+-----------+------------+---------+
|https://www.advoc...|Christmas Shoppin...|  850|   850| 0.2666246294975281|  nolang|-3604776403351267688|0.039626352|2.7811527E-4| 8.352225|
|https://img.freep...|Cremiger hüttenkä...|  626|   415| 0.2779805064201355|      de|  571058205933102479| 0.06443764|0.0011278689| 8.013723|
|https://zakarpat....|A. Landovska Brea...|  300|   344|0.28194114565849304|      sv|-8455279219565229056|0.023449354|2.2380891E-5|8.0435295|
|https://www.povar...|Рецепт: Воздушные...|  330|   220| 0.2608181834220886|      ru|-3137830403123787972| 0.33741534|0.0047357082| 8.149608|
|http:

In [6]:
df.count()

8070941

In [47]:
"WIDTH" in df.schema.names

True

In [7]:
if "WIDTH" in df.schema.names:
    df = df.filter((df.WIDTH >= filter_width) & (df.HEIGHT >= filter_height) & (df.LANGUAGE == 'en'))
else:
    df = df.filter((df.width >= filter_width) & (df.height >= filter_height) & (df.language == 'en'))
df = df.orderBy(rand()) # this line is important to have a shuffled dataset
#df.repartition(num_repartitions).write.parquet(output_dir)

In [8]:
df.count()

947191

In [9]:
#min(df.select(df.WIDTH).collect())
from pyspark.sql import functions
df.select(functions.min("WIDTH")).collect()
df.select(functions.min("HEIGHT")).collect()

[Row(min(HEIGHT)=512)]

In [59]:
947191/8070941

0.11735818661045844

In [92]:
df.show(10)

23/07/14 08:30:30 ERROR Executor: Exception in task 4.0 in stage 297.0 (TID 1648)
org.apache.spark.SparkFileNotFoundException: File file:/home/yx/datasets/diffusion/laion_art/laion-art.parquet does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:794)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:234)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:293)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:125)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:594)
	at

Py4JJavaError: An error occurred while calling o126.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 12 in stage 297.0 failed 1 times, most recent failure: Lost task 12.0 in stage 297.0 (TID 1656) (10.90.53.110 executor driver): org.apache.spark.SparkFileNotFoundException: File file:/home/yx/datasets/diffusion/laion_art/laion-art.parquet does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:794)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:234)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:293)
	at org.apache.sp

In [23]:
#abn_img_url = "https://oyster.ignimgs.com/wordpress/stg.ign.com/2015/01/carter_010615_1600.jpg?width=375&crop=16%3A9&quality=20&dpr=0.05"
abn_img_url = "http://www.shedswarehouse.com/images/Large/13713.jpg"
#df_abn = df.filter((df.WIDTH == 1))
df_abn = df.filter((df.URL == abn_img_url))
df_abn.show()
print(df_abn.select("URL").collect())
print(df_abn.select("TEXT").collect())

+--------------------+--------------------+-----+------+-----------------+--------+--------------------+------------+------------+---------+
|                 URL|                TEXT|WIDTH|HEIGHT|       similarity|LANGUAGE|                hash|  pwatermark|     punsafe|aesthetic|
+--------------------+--------------------+-----+------+-----------------+--------+--------------------+------------+------------+---------+
|http://www.shedsw...|Oxford insulated ...| 1024|  1024|0.351455956697464|      en|-2593509028067492070|0.0010399926|0.0045304894| 8.082609|
+--------------------+--------------------+-----+------+-----------------+--------+--------------------+------------+------------+---------+

[Row(URL='http://www.shedswarehouse.com/images/Large/13713.jpg')]


[Row(TEXT='Oxford insulated garden rooms avon for Garden room definition')]


In [49]:
# no lang filter
df.count()

2378491

In [50]:
keep_ratio = 2378491/8070941
keep_ratio

0.2946981027367193

In [51]:
df.schema.names

['URL',
 'TEXT',
 'WIDTH',
 'HEIGHT',
 'similarity',
 'LANGUAGE',
 'hash',
 'pwatermark',
 'punsafe',
 'aesthetic']

### check images download result



In [87]:
spark2 = SparkSession.builder.config("spark.driver.memory", "1G") .master("local[4]").appName('spark-stats2').getOrCreate() 
df_download = spark2.read.parquet('/data3/datasets/laion_art_filtered/00000.parquet')

23/07/14 07:52:09 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
23/07/14 07:52:09 ERROR Executor: Exception in task 0.0 in stage 294.0 (TID 1611)
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:396)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:422)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1(ParquetFileFormat.scala:472)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1$adapted(ParquetFileFormat.scala:464)
	at org.apache.spark.sql.execution.datasources.SchemaMergeUtils$.$anonfun$mergeSchemasInParallel$2(SchemaMergeUtils.scala:79)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartiti

Py4JJavaError: An error occurred while calling o239.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 294.0 failed 1 times, most recent failure: Lost task 0.0 in stage 294.0 (TID 1611) (10.90.53.110 executor driver): org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:396)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:422)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1(ParquetFileFormat.scala:472)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1$adapted(ParquetFileFormat.scala:464)
	at org.apache.spark.sql.execution.datasources.SchemaMergeUtils$.$anonfun$mergeSchemasInParallel$2(SchemaMergeUtils

## LAION 2b-en ori

In [19]:
ori  = load_dataset("parquet", data_dir="/home/yx/datasets/diffusion/laion_2b_en")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/default-d308bf0fa364f6e6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
len(ori['train'])

18141938

In [21]:
ori

DatasetDict({
    train: Dataset({
        features: ['SAMPLE_ID', 'URL', 'TEXT', 'HEIGHT', 'WIDTH', 'LICENSE', 'NSFW', 'similarity'],
        num_rows: 18141938
    })
})

In [23]:
ori['train'].__getitem__(0)

{'SAMPLE_ID': 2641080021034,
 'URL': 'https://cdn.shopify.com/s/files/1/0017/3621/2538/products/blue-beach-umbrellas-point-of-rocks-crescent-beach-siesta-key-shawn-mcloughlin_32d72f5b-5e55-42f9-bfcf-d6fa8d239beb_300x300.jpg?v=1524171284',
 'TEXT': 'Blue Beach Umbrellas, Point Of Rocks, Crescent Beach, Siesta Key - Spiral Notebook',
 'HEIGHT': 231,
 'WIDTH': 300,
 'LICENSE': '?',
 'NSFW': 'UNLIKELY',
 'similarity': 0.3955616354942322}

In [62]:
import os

os.listdir(".")

['fusion_result.json',
 'rank_0',
 "':w",
 'test_load_laion.ipynb',
 'rank_1',
 'tools',
 'arch_params.txt',
 'download_laion_imgs.sh',
 'demo.md',
 'utils',
 'output_sd1x',
 'schedulers.md',
 'params_ori.txt',
 'train_text_to_image.py',
 'laion_check_and_convert.py',
 'laion_filter_metadata.py',
 '.laion_train.md.swp',
 'nohup.out',
 '.laion_filter_metadata.py.swp',
 'bpe_simple_vocab_16e6.txt.gz',
 'test_ge.py',
 'infer_ge.sh',
 'Untitled.ipynb',
 '.laion_check_and_convert.py.swp',
 'arch.txt',
 'demo_res.ipynb',
 'requirements.txt',
 'test_lora_ori_net.ckpt',
 '.ipynb_checkpoints',
 'output',
 'test_lora_tp_after_ft.ckpt',
 'train_autoencoder.py',
 'lora_finetune.md',
 'test_load_laion.py',
 '.download_laion_imgs.sh.swp',
 'scripts',
 'kernel_meta_temp_f87b28a10d8fafc3e16fe1b9ae8d913dadc9bee95d1a8140e304f8db9e4d8086',
 'ckpt_ana_res.txt',
 'models',
 'laion_train.md',
 'ldm',
 'bak',
 '__pycache__',
 'analyze_ckpt.py',
 'convert_dataset.py',
 'laion-art',
 ';',
 'arch_lora.txt',
 't